# Given dataset
Suppose you are a DA in an e-wallet company, and you need to analyze the following datasets:
- payment_report.csv (monthly payment volume of products)
- product.csv (product information)
- transactions.csv (transactions information)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
path1 = '/content/drive/MyDrive/payment_report.csv'
payment_report = pd.read_csv(path1, encoding='utf-8')

path2 = '/content/drive/MyDrive/product.csv'
product = pd.read_csv(path2, encoding='utf-8')

path3 = '/content/drive/MyDrive/transactions.csv'
transactions = pd.read_csv(path3, encoding='utf-8')

In [4]:
payment_report

,report_month,payment_group,product_id,source_id,volume
0,2023-01,payment,12,45,624110375
1,2023-01,payment,17,45,335715113
2,2023-01,payment,18,45,737784466
3,2023-01,payment,19,45,120963069
4,2023-01,payment,20,45,319653158
...,...,...,...,...,...
914,2023-04,payment,15067,45,1504000
915,2023-04,refund,1976,37,3542271587
916,2023-04,refund,1976,38,13831708189
917,2023-04,refund,1976,39,1905435543


In [ ]:
product

,product_id,category,team_own
0,17,PXXXXXB,ASD
1,18,PXXXXXB,ASD
2,20,PXXXXXB,ASD
3,287,PXXXXXB,ASD
4,372,PXXXXXB,ASD
...,...,...,...
487,321,PXXXXXV,ASD
488,322,PXXXXXV,ASD
489,341,PXXXXXV,ASD
490,342,PXXXXXV,ASD


In [ ]:
transactions

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp
0,3002692434,5,100000,24,1,10199794.0,199794.0,NaN,1682932054455
1,3002692437,305,20000,2,1,14022211.0,14022211.0,NaN,1682932054912
2,3001960110,7255,48605,22,1,NaN,10530940.0,NaN,1682932055000
3,3002680710,2270,1500000,2,1,10059206.0,59206.0,NaN,1682932055622
4,3002680713,2275,90000,2,1,10004711.0,4711.0,NaN,1682932056197
...,...,...,...,...,...,...,...,...,...
1323997,3003723030,305,20000,2,1,24524311.0,NaN,NaN,1683035672634
1323998,3003723033,2270,100000,2,1,10277242.0,277242.0,NaN,1683035672876
1323999,3003723036,2270,100000,2,1,10144599.0,144599.0,NaN,1683035672892
1324000,3003723039,5,400,22,1,10028007.0,21013762.0,NaN,1683035672896


# Part I: EDA

## Merge payment_report.csv with product.csv

In [ ]:
payment_enriched = pd.merge(payment_report,product,how='left',on='product_id')
payment_enriched

,report_month,payment_group,product_id,source_id,volume,category,team_own
0,2023-01,payment,12,45,624110375,PXXXXXT,ASD
1,2023-01,payment,17,45,335715113,PXXXXXB,ASD
2,2023-01,payment,18,45,737784466,PXXXXXB,ASD
3,2023-01,payment,19,45,120963069,PXXXXXM2,ASD
4,2023-01,payment,20,45,319653158,PXXXXXB,ASD
...,...,...,...,...,...,...,...
914,2023-04,payment,15067,45,1504000,PXXXXXR,ASL
915,2023-04,refund,1976,37,3542271587,NaN,NaN
916,2023-04,refund,1976,38,13831708189,NaN,NaN
917,2023-04,refund,1976,39,1905435543,NaN,NaN


## Missing data

In [ ]:
payment_enriched.isnull().sum()

,0
report_month,0
payment_group,0
product_id,0
source_id,0
volume,0
category,22
team_own,22


In [ ]:
transactions.isnull().sum()

,0
transaction_id,0
merchant_id,0
volume,0
transType,0
transStatus,0
sender_id,49059
receiver_id,164795
extra_info,1317907
timeStamp,0


### Note
- No null from payment_enriched
- 49059 NaN from transactions[sender_id]
- 164795 NaN from transactions[receiver_id]
- 1317907 NaN from transactions[extra_info]

Solutions:
- Ignore NaN from extra_info
- Delete if both sender_id and receiver_id are missing



In [ ]:
# Delete if both sender_id and receiver_id are missing
null_both = transactions[transactions['sender_id'].isnull() & transactions['receiver_id'].isnull()]
transactions_cleaned = transactions.dropna(subset=['sender_id','receiver_id'], how='all')

Do not know the reasons for missing values in receiver_id, sender_id at the moment
- Replace NaN = -1

In [ ]:
transactions_cleaned['sender_id'].fillna(-1, inplace=True)
transactions_cleaned['receiver_id'].fillna(-1, inplace=True)
transactions_cleaned.isnull().sum()

<ipython-input-147-3500ba6b8fb4>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions_cleaned['sender_id'].fillna(-1, inplace=True)
<ipython-input-147-3500ba6b8fb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions_cleaned['sender_id'].fillna(-1, inplace=True)
<ipython-input-147-3500ba6b8fb4>:2: FutureWarning: A value is t

,0
transaction_id,0
merchant_id,0
volume,0
transType,0
transStatus,0
sender_id,0
receiver_id,0
extra_info,1315320
timeStamp,0


## Check duplicates

In [ ]:
transactions_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1321415 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1321415 non-null  int64  
 1   merchant_id     1321415 non-null  int64  
 2   volume          1321415 non-null  int64  
 3   transType       1321415 non-null  int64  
 4   transStatus     1321415 non-null  int64  
 5   sender_id       1321415 non-null  float64
 6   receiver_id     1321415 non-null  float64
 7   extra_info      6095 non-null     object 
 8   timeStamp       1321415 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 100.8+ MB


In [ ]:
transactions_cleaned.nunique()

,0
transaction_id,1321387
merchant_id,628
volume,49341
transType,7
transStatus,35
sender_id,173558
receiver_id,200292
extra_info,2037
timeStamp,1314014


There are some duplicated values in transactions_cleaned -> Drop these duplicates


In [ ]:
transactions_final = transactions_cleaned.drop_duplicates('transaction_id')

## Incorrect data type & data value

In [ ]:
payment_enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   919 non-null    object
 1   payment_group  919 non-null    object
 2   product_id     919 non-null    int64 
 3   source_id      919 non-null    int64 
 4   volume         919 non-null    int64 
 5   category       897 non-null    object
 6   team_own       897 non-null    object
dtypes: int64(3), object(4)
memory usage: 50.4+ KB


In [ ]:
payment_enriched.describe()

,product_id,source_id,volume
count,919.000000,919.000000,9.190000e+02
mean,1192.517954,44.875952,1.978574e+08
std,1293.463329,0.910995,8.367595e+08
min,3.000000,37.000000,5.500000e+03
25%,640.000000,45.000000,1.250000e+06
50%,1059.000000,45.000000,7.982015e+06
75%,1585.000000,45.000000,5.447599e+07
max,15067.000000,45.000000,1.383171e+10


In [ ]:
transactions_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1321387 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1321387 non-null  int64  
 1   merchant_id     1321387 non-null  int64  
 2   volume          1321387 non-null  int64  
 3   transType       1321387 non-null  int64  
 4   transStatus     1321387 non-null  int64  
 5   sender_id       1321387 non-null  float64
 6   receiver_id     1321387 non-null  float64
 7   extra_info      6095 non-null     object 
 8   timeStamp       1321387 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 100.8+ MB


In [ ]:
transactions_final.describe()

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,timeStamp
count,1.321387e+06,1.321387e+06,1.321387e+06,1.321387e+06,1.321387e+06,1.321387e+06,1.321387e+06,1.321387e+06
mean,3.002233e+09,2.462103e+03,2.382381e+05,6.934112e+00,-1.193622e+01,9.975608e+07,1.828956e+08,1.683130e+12
std,1.043623e+07,3.306412e+03,9.644103e+05,7.397025e+00,5.558634e+01,6.126611e+08,8.725972e+08,1.707796e+08
min,3.000000e+09,5.000000e+00,1.000000e+00,2.000000e+00,-1.333000e+03,-1.000000e+00,-6.300000e+01,1.682874e+12
25%,3.001121e+09,3.050000e+02,1.000000e+04,2.000000e+00,1.000000e+00,1.004134e+07,4.201400e+04,1.682994e+12
50%,3.002200e+09,2.250000e+03,3.000000e+04,2.000000e+00,1.000000e+00,1.057732e+07,3.531248e+06,1.683097e+12
75%,3.003255e+09,2.270000e+03,1.000000e+05,8.000000e+00,1.000000e+00,2.100212e+07,2.451698e+07,1.683269e+12
max,6.000066e+09,1.625250e+05,7.869148e+07,3.000000e+01,1.000000e+00,6.993439e+09,2.100000e+10,1.683479e+12


No incorrect values in both datasets


# Part II: Data Wrangling

## Using payment_report.csv & product.csv


### Q1: Top 3 product_ids with the highest volume

In [ ]:
volume_by_product = payment_enriched.groupby('product_id',as_index=False).agg({'volume':'sum'})
top_3_product = volume_by_product.nlargest(3,'volume')
top_3_product

,product_id,volume
279,1976,61797583647
43,429,14667676567
38,372,13713658515


### Q2: Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?


In [ ]:
unique_team_by_product = payment_enriched.groupby('product_id',as_index=False).agg({'team_own':'nunique'})
unique_team_by_product[unique_team_by_product['team_own'] !=1]

,product_id,team_own
0,3,0
279,1976,0
308,10033,0


### Q3: Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.


In [ ]:
# Edit date_time value
payment_enriched['report_month'] = pd.to_datetime(payment_enriched['report_month'])
payment_enriched['year'] = payment_enriched['report_month'].dt.year
payment_enriched['quarter'] = payment_enriched['report_month'].dt.quarter

# Filter data since Q2.2023
from_q2_2023 = payment_enriched[(payment_enriched['year']>=2023) & (payment_enriched['quarter']>=2)]

# Total volume by team
team_performance = from_q2_2023.groupby('team_own').agg({'volume':'sum'})

# Find the team with the lowest total volume
lowest_performance_team = team_performance['volume'].idxmin()
print(f"The lowest performing team since Q2.20023 is:{lowest_performance_team}")

# Find the category that contributes the least to that team
## Filter from the dataset the lowest performace team
data_team = from_q2_2023[from_q2_2023['team_own'] == lowest_performance_team]
## Group volume by category
category_contribute = data_team.groupby('category').agg({'volume':'sum'})
## The least contribute
lowest_category_contribute = category_contribute['volume'].idxmin()
print(f"Category that contributes the least to {lowest_performance_team} is:{lowest_category_contribute}")


The lowest performing team since Q2.20023 is:APS
Category that contributes the least to APS is:PXXXXXE


### Q4: Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?

In [ ]:
# Filter payment_group = refund
refund_payment = payment_enriched[payment_enriched['payment_group'] == 'refund']

# Total volume group by source_id
soure_id_contribute = refund_payment.groupby('source_id')['volume'].sum()

# Highest contribution source_id
highest_source_id_contribute = soure_id_contribute.idxmax()
print(f"Source ID with the highest contribution to refund trasactions is: {highest_source_id_contribute}")

Source ID with the highest contribution to refund trasactions is: 38


## Using transactions.csv

### Q5: Define type of transactions (‘transaction_type’) for each row, given:
- transType = 2 & merchant_id = 1205: Bank Transfer Transaction
- transType = 2 & merchant_id = 2260: Withdraw Money Transaction
- transType = 2 & merchant_id = 2270: Top Up Money Transaction
- transType = 2 & others merchant_id: Payment Transaction
- transType = 8, merchant_id = 2250: Transfer Money Transaction
- transType = 8 & others merchant_id: Split Bill Transaction
- Remained cases are invalid transactions


In [ ]:
def define_transaction_type(row):
  if row['transType'] == 2 and row['merchant_id'] == 1205:
    return 'Bank Transfer Transaction'
  elif row['transType'] == 2 and row['merchant_id'] == 2260:
    return 'Withdraw Money Transaction'
  elif row['transType'] == 2 and row['merchant_id'] == 2270:
    return 'Top Up Money Transaction'
  elif row['transType'] == 2:
    return 'Payment Transaction'
  elif row['transType'] == 8 and row['merchant_id'] == 2250:
    return 'Transfer Money Transaction'
  elif row['transType'] == 8:
    return 'Split Bill Transaction'
  else:
    return 'Invalid Transactions'

transactions['transaction_types'] = transactions.apply(define_transaction_type, axis =1)
transactions

,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,extra_info,timeStamp,transaction_types
0,3002692434,5,100000,24,1,10199794.0,199794.0,NaN,1682932054455,Invalid Transactions
1,3002692437,305,20000,2,1,14022211.0,14022211.0,NaN,1682932054912,Payment Transaction
2,3001960110,7255,48605,22,1,NaN,10530940.0,NaN,1682932055000,Invalid Transactions
3,3002680710,2270,1500000,2,1,10059206.0,59206.0,NaN,1682932055622,Top Up Money Transaction
4,3002680713,2275,90000,2,1,10004711.0,4711.0,NaN,1682932056197,Payment Transaction
...,...,...,...,...,...,...,...,...,...,...
1323997,3003723030,305,20000,2,1,24524311.0,NaN,NaN,1683035672634,Payment Transaction
1323998,3003723033,2270,100000,2,1,10277242.0,277242.0,NaN,1683035672876,Top Up Money Transaction
1323999,3003723036,2270,100000,2,1,10144599.0,144599.0,NaN,1683035672892,Top Up Money Transaction
1324000,3003723039,5,400,22,1,10028007.0,21013762.0,NaN,1683035672896,Invalid Transactions


### Q6: Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.

In [ ]:
# Excluding invalid transactions
filter_transactions = transactions[transactions['transaction_types'] != 'Invalid Transactions']

# Number of transactions, volume, senders and receivers group by transaction type
transaction_type_summary = filter_transactions.groupby('transaction_types').agg(
    num_transactions =('transaction_id','count'),
    total_volume = ('volume','sum'),
    num_senders = ('sender_id','nunique'),
    num_receivers =('receiver_id','nunique')).reset_index()
transaction_type_summary

,transaction_types,num_transactions,total_volume,num_senders,num_receivers
0,Bank Transfer Transaction,37879,50605806190,23156,9271
1,Payment Transaction,398677,71851515181,139583,113298
2,Split Bill Transaction,1376,4901464,1323,572
3,Top Up Money Transaction,290502,108606478829,110409,110409
4,Transfer Money Transaction,341177,37033171492,39021,34585
5,Withdraw Money Transaction,33725,23418181420,24814,24814
